#### Fetching data

In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 9.6 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 8.2 MB/s  0:00:01m0:00:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install BeautifulSoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [BeautifulSoup4]m [BeautifulSoup4]
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install selenium

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 9.4 MB/s  0:00:016m0:00:0100:01
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Attempting uninstall: typing_extensions━━━━━━━━━━━━━━━━━━━━━━━━━  1/11 [websocket-client]
    Found existing installation: typing_extensions 4.12.2━━━━━  1/11 [websocket-client]
    Uninstalling typing_extensions-4.12.2:━━━━━━━━━━━━━━━━━━━━  1/11 [websocket-client]
      Successfully uninstalled typing_extensions-4.12.2━━━━━━━━━━━  2/11 [typing_extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [selenium]/11 [selenium]ocket]ns]
Note: you may need to restart the kernel to use updated packages.


In [35]:
import re
import os
import pandas as pd
import requests
import string
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# --- SELENIUM IMPORTS ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
BASE_URL = "https://econpapers.repec.org"
JEL_LETTERS = ["A", "B", "C"]   # test
LIMIT_PER_CATEGORY = 20

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "From": "researcher@university.edu"
}

# ---------------------------------------------------------
# SELENIUM — LIENS
# ---------------------------------------------------------
def get_links_with_selenium(jel_letter):

    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )

    paper_links = []

    url = (
        "https://econpapers.repec.org/scripts/search.pf"
        f"?jel={jel_letter}*&ni=10%20years&inpage=1000"
    )

    driver.get(url)
    wait = WebDriverWait(driver, 10)

    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//input[@type='SUBMIT']")
    )).click()

    Select(wait.until(
        EC.presence_of_element_located((By.ID, "inpage1"))
    )).select_by_value(str(LIMIT_PER_CATEGORY))

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    links = []
    for a in soup.find_all("a", href=re.compile(r"/paper/|/article/")):
        href = a["href"]
        if "scripts" in href or "pers" in href:
            continue
        links.append(urljoin(BASE_URL, href))

    return list(dict.fromkeys(links))

# ---------------------------------------------------------
# SCRAPING MÉTADONNÉES
# ---------------------------------------------------------
def get_paper_details(url, jel_cat):

    try:
        soup = BeautifulSoup(
            requests.get(url, headers=HEADERS, timeout=10).text,
            "html.parser"
        )

        # --- TITRE ---
        title = soup.find("h1", class_="colored").get_text(strip=True)

        # --- ANNÉE ---
        year = None
        date = soup.find("b", string=re.compile("Date:"))
        if date:
            m = re.search(r"\d{4}", date.next_sibling or "")
            if m:
                year = int(m.group())

        # --- AUTEURS (SOURCE FIABLE) ---
        authors = [
            m["content"].replace(",", "")
            for m in soup.find_all("meta", {"name": "citation_author"})
        ]

        if not authors:
            authors = ["Voir texte"]
        # --- JOURNAL ---
        journal = None
        journal_meta = soup.find("meta", {"name": "citation_journal_title"})
        if journal_meta:
            journal = journal_meta.get("content", None)

        # --- AFFILIATIONS ---
        affil = soup.find("span", id="contact")
        affiliations = affil.get_text(" ", strip=True) if affil else None

        # --- TYPE ---
        pub_type = "Journal Article" if "/article/" in url else "Working Paper"

        return {
            "JEL Subject": jel_cat,
            "Title": title,
            "Author(s)": "; ".join(authors),
            "Journal": journal, 
            "Year": year,
            "Type": pub_type,
            "Affiliations": affiliations,
            "URL": url
        }

    except Exception:
        return None

# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------
def main(csv_filename):

    data = []

    for jel in JEL_LETTERS:
        links = get_links_with_selenium(jel)
        for link in links:
            d = get_paper_details(link, jel)
            if d:
                data.append(d)

    df = pd.DataFrame(data).drop_duplicates("URL")
    df.to_csv(csv_filename, index=False)

# ---------------------------------------------------------
# RUN
# ---------------------------------------------------------
if __name__ == "__main__":

    out = "RePEc_Final_Dataset_Corrected.csv"
    if os.path.exists(out):
        os.remove(out)

    main(out)

#### Data cleaning

In [36]:
df = pd.read_csv("RePEc_Final_Dataset_Corrected.csv")
import pandas as pd
import numpy as np

def clean_affiliation(text):
    if pd.isna(text):
        return np.nan
    
    # Si le format est "Auteur: Affiliation"
    if ":" in text:
        return text.split(":", 1)[1].strip()
    
    return text.strip()

df["Affiliations"] = df["Affiliations"].apply(clean_affiliation)
df = df.iloc[2:].reset_index(drop=True)
df.head(20)

,JEL Subject,Title,Author(s),Journal,Year,Type,Affiliations,URL
0,A,Preparing students for careers using business ...,Nielsen Erland Hejn; Nielsen Steen,NaN,2020.0,Working Paper,Department of Economics and Business Economics...,https://econpapers.repec.org/paper/aahaarhec/2...
1,A,"Measuring Democracy - Eight indices: Polity, F...",Paldam Martin,NaN,2021.0,Working Paper,Department of Economics and Business Economics...,https://econpapers.repec.org/paper/aahaarhec/2...
2,A,Oeconstudiet og den ÃÂ¸konomiske faggruppe ve...,Hylleberg Svend,NaN,2023.0,Working Paper,Department of Economics and Business Economics...,https://econpapers.repec.org/paper/aahaarhec/2...
3,A,Digital Tools in the Educational Environment E...,Andra Diaconescu,Research & Education,2024.0,Journal Article,"Politehnica University of Timisoara, Faculty o...",https://econpapers.repec.org/article/aaijournl...
4,A,On the Gender Diversity of Research Teams in E...,Biermann Marcus,AEA Papers and Proceedings,2023.0,Journal Article,NaN,https://econpapers.repec.org/article/aeaapandp...
5,A,Messages That Foster a Sense of Belonging Impr...,Forcada Sara Avila,AEA Papers and Proceedings,2023.0,Journal Article,NaN,https://econpapers.repec.org/article/aeaapandp...
6,A,Parenthood and Academic Career Trajectories,Lassen Anne Sophie; IvandiÄ Ria,AEA Papers and Proceedings,2024.0,Journal Article,NaN,https://econpapers.repec.org/article/aeaapandp...
7,A,Impact versus Inclusion in the Economics Profe...,Bansak Cynthia; Dunn Wendy; Meade Ellen; Starr...,AEA Papers and Proceedings,2024.0,Journal Article,NaN,https://econpapers.repec.org/article/aeaapandp...
8,A,Teaching-Track Economists: A Canadian Perspective,Murdock Jennifer; Cohen Avi,AEA Papers and Proceedings,2024.0,Journal Article,NaN,https://econpapers.repec.org/article/aeaapandp...
9,A,"Male Is a Gender, Too: A Review of Why Gender ...",Nelson Julie,Journal of Economic Literature,2016.0,Journal Article,NaN,https://econpapers.repec.org/article/aeajeclit...


In [31]:
df_voir_texte = df[df["Author(s)"] == "Voir texte"]
len(df_voir_texte)


35

In [32]:
df_voir_texte["URL"].head(3).tolist()


['https://econpapers.repec.org/article/aeaapandp/v_3a113_3ay_3a2023_3ap_3a473-76.htm',
 'https://econpapers.repec.org/article/aeaapandp/v_3a113_3ay_3a2023_3ap_3a514-18.htm',
 'https://econpapers.repec.org/article/aeaapandp/v_3a114_3ay_3a2024_3ap_3a238-42.htm']